In [ ]:
import sys
from pathlib import Path

Jupyter server should be run at the notebook directory, so the output of the following cell would be the project root:

In [ ]:
project_root = str(Path.cwd().resolve().parent)
print(f"Project root: {project_root}")

In [ ]:
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
%load_ext autoreload
# Reload all modules imported with %aimport every time before executing the Python code typed.
%autoreload 1
%aimport src.analysis.visualize_columns, src.database.database_connection

In [ ]:
from src.analysis import DataVisualizer
from src.preprocess import preprocess_data
from src.database import DatabaseConnection

In [ ]:
db_connection = DatabaseConnection("../data/slurm_data.db")
jobs_df = db_connection.fetch_all_jobs()

In [ ]:
clean_jobs_df = preprocess_data(jobs_df, min_elapsed_seconds=600)
clean_jobs_df

In [ ]:
visualizer = DataVisualizer(clean_jobs_df.sample(10000, random_state=42))

In [ ]:
visualizer.visualize_columns(columns=["Elapsed", "TimeLimit"], output_dir_path=Path("../data/visualizations"))